In [225]:
from tabulate import tabulate
import random

In [226]:
from random import randint, choice, random
from math import exp
from itertools import count
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors
from collections import deque
from heapq import heappop, heappush
from tabulate import tabulate
import random


In [227]:
class Problem:
    def __init__(self):
        self.init_state = None

    def actions(self, state):
        raise NotImplementedError

    def result(self, state, action):
        raise NotImplementedError

    def goal_test(self, state):
        raise NotImplementedError

    def step_cost(self, state, action):
        raise NotImplementedError

In [228]:
class Node:
    def __init__(self, state, parent, action, path_cost):
        self.state = state
        self.parent = parent
        self.action = action
        self.path_cost = path_cost

    @classmethod
    def root(cls, init_state):
        return cls(init_state, None, None, 0)

    @classmethod
    def child(cls, problem, parent, action):
        return cls(
            problem.result(parent.state, action),
            parent,
            action,
            parent.path_cost + problem.step_cost(parent.state, action))

def solution(node):
    actions = []
    cost = node.path_cost
    while node.parent is not None:
        actions.append(node.action)
        node = node.parent
    actions.reverse()
    return actions, cost

In [229]:
from shutil import get_terminal_size
terminal_width, _ = get_terminal_size()

visualizers = {}

def _default_visualizer(_, state):
    print(state)

class Visualizer:

    def __init__(self, problem):
        self.problem = problem
        self.counter = 0

    def visualize(self, frontier):
        self.counter += 1
        print(f'Frontier at step {self.counter}')
        for node in frontier:
            print()
            visualizers.get(type(self.problem), _default_visualizer)(self.problem, node.state)
        print('-' * terminal_width)

In [230]:
class SlidingPuzzle(Problem):

    def __init__(self, init_state, goal_state):
        assert init_state.count(' ') == 1
        assert goal_state.count(' ') == 1
        self.init_state = tuple(init_state)
        self._goal_state = tuple(goal_state)
        self._action_values = {'up': -3, 'down': +3, 'left': -1, 'right': +1}

    def actions(self, state):
        index = state.index(' ')
        possible_moves = []
        if index // 3 > 0:
            possible_moves.append('up')
        if index // 3 < 2:
            possible_moves.append('down')
        if index % 3 > 0:
            possible_moves.append('left')
        if index % 3 < 2:
            possible_moves.append('right')
        return possible_moves

    def result(self, state, action):
        def swap(t, i, j):
            l = list(t)
            l[i], l[j] = l[j], l[i]
            return tuple(l)
        index = state.index(' ')
        return swap(state, index, index + self._action_values[action])

    def goal_test(self, state):
        return state == self._goal_state

    def step_cost(self, state, action):
        return 1

    def heuristic(self, state):
        """Simple heuristic: number of tiles out of place compared to the goal state."""
        return sum(1 for i in range(len(state)) if state[i] != self.goal_state[i])

def sliding_puzzle_visualizer(problem, state):
    for i in range(0, 9, 3):
        print(' ' + ' '.join(state[i:i + 3]) + ' ')

visualizers[SlidingPuzzle] = sliding_puzzle_visualizer

In [231]:
problem = SlidingPuzzle('12345678 ', '123 56478')

In [232]:
from collections import deque

def breadth_first_graph_search(problem, verbose=False):
    """
    Implements BFS for graph search with an optional verbose output.
    """
    node = Node.root(problem.init_state)  # Create the root node from the initial state
    if problem.goal_test(node.state):  # Check if the initial state is the goal
        return solution(node), 1  # Return immediately if start is the goal

    frontier = deque([node])  # Initialize the frontier with the root node
    explored = set()  # Set to keep track of explored nodes
    max_frontier_size = 1  # Initialize max frontier size

    if verbose:
        visualizer = Visualizer(problem)  # Optional visualizer for debugging
        visualizer.visualize(frontier)

    while frontier:  # While there are nodes to explore
        node = frontier.popleft()  # Dequeue the first node in the frontier
        explored.add(node.state)  # Mark the node as explored

        # Expand the frontier with the children
        for action in problem.actions(node.state):  # Get possible actions from the current state
            child = Node.child(problem, node, action)  # Generate child node
            if child.state not in explored and child not in frontier:  # Check if child is unexplored
                if problem.goal_test(child.state):  # Check if the child is the goal
                    return solution(child), max_frontier_size  # Return the solution and max frontier size
                frontier.append(child)  # Add child to the frontier

        # Track the maximum size of the frontier
        max_frontier_size = max(max_frontier_size, len(frontier))

        if verbose:
            visualizer.visualize(frontier)  # Visualize the current state of the frontier

    return None, max_frontier_size  # Return None if no solution is found


In [233]:
# Running BFS graph search with verbose output
actions, cost = breadth_first_graph_search(problem, verbose=True)
print(f"Solution(actions, step_cost): {actions}, \nMax Frontier Size: {cost}")

Frontier at step 1

 1 2 3 
 4 5 6 
 7 8   
----------------------------------------------------------------------------------------------------
Frontier at step 2

 1 2 3 
 4 5   
 7 8 6 

 1 2 3 
 4 5 6 
 7   8 
----------------------------------------------------------------------------------------------------
Frontier at step 3

 1 2 3 
 4 5 6 
 7   8 

 1 2   
 4 5 3 
 7 8 6 

 1 2 3 
 4   5 
 7 8 6 
----------------------------------------------------------------------------------------------------
Frontier at step 4

 1 2   
 4 5 3 
 7 8 6 

 1 2 3 
 4   5 
 7 8 6 

 1 2 3 
 4   6 
 7 5 8 

 1 2 3 
 4 5 6 
   7 8 
----------------------------------------------------------------------------------------------------
Frontier at step 5

 1 2 3 
 4   5 
 7 8 6 

 1 2 3 
 4   6 
 7 5 8 

 1 2 3 
 4 5 6 
   7 8 

 1   2 
 4 5 3 
 7 8 6 
----------------------------------------------------------------------------------------------------
Frontier at step 6

 1 2 3 
 4   6 
 7 5 8 

 1 2 

In [234]:
def depth_first_graph_search(problem, verbose=False):
    """
    Search the deepest nodes in the search tree first.
    Search through the successors of a problem to find a goal.
    Does not get trapped by loops.
    """
    node = Node.root(problem.init_state)
    frontier = [node]  # Stack (LIFO)
    explored = set()
    max_frontier_size = 1  # Initialize max frontier size

    if verbose:
        visualizer = Visualizer(problem)
        visualizer.visualize(frontier)

    while frontier:
        node = frontier.pop()
        if problem.goal_test(node.state):
            return solution(node), max_frontier_size  # Return solution and cost properly

        explored.add(node.state)
        for action in problem.actions(node.state):
            child = Node.child(problem, node, action)
            if child.state not in explored and child not in frontier:
                frontier.append(child)

        # Track the maximum size of the frontier
        max_frontier_size = max(max_frontier_size, len(frontier))

        if verbose:
            visualizer.visualize(frontier)

    return None, max_frontier_size

In [235]:
# Running DFS graph search with verbose output
actions, cost = depth_first_graph_search(problem, verbose=True)
print(f"Solution(actions, step_cost): {actions}, \nMax Frontier Size: {cost}")

Frontier at step 1

 1 2 3 
 4 5 6 
 7 8   
----------------------------------------------------------------------------------------------------
Frontier at step 2

 1 2 3 
 4 5   
 7 8 6 

 1 2 3 
 4 5 6 
 7   8 
----------------------------------------------------------------------------------------------------
Frontier at step 3

 1 2 3 
 4 5   
 7 8 6 

 1 2 3 
 4   6 
 7 5 8 

 1 2 3 
 4 5 6 
   7 8 
----------------------------------------------------------------------------------------------------
Frontier at step 4

 1 2 3 
 4 5   
 7 8 6 

 1 2 3 
 4   6 
 7 5 8 

 1 2 3 
   5 6 
 4 7 8 
----------------------------------------------------------------------------------------------------
Solution(actions, step_cost): (['left', 'left', 'up'], 3), 
Max Frontier Size: 3


In [236]:
import sys
from collections import deque

def depth_limited_search(problem, limit=50, verbose=False, visualizer=None):
    """Performs a depth-limited search up to a specified limit."""
    def recursive_dls(node, problem, limit):
        if problem.goal_test(node.state):
            return node
        elif limit == 0:
            return 'cutoff'
        else:
            cutoff_occurred = False
            children = [Node.child(problem, node, action) for action in problem.actions(node.state)]
            if verbose and visualizer:
                visualizer.visualize(children)
            for child in children:
                result = recursive_dls(child, problem, limit - 1)
                if result == 'cutoff':
                    cutoff_occurred = True
                elif result is not None:
                    return result
            return 'cutoff' if cutoff_occurred else None

    # Body of depth_limited_search:
    return recursive_dls(Node.root(problem.init_state), problem, limit)


def iterative_deepening_search(problem, verbose=False):
    """Performs iterative deepening search with verbose output."""
    max_frontier_size = 1  # Track the max size of the frontier
    visualizer = Visualizer(problem) if verbose else None

    for depth in range(sys.maxsize):
        result = depth_limited_search(problem, limit=depth, verbose=verbose, visualizer=visualizer)

        if result != 'cutoff':
            actions = solution(result)
            return actions, max_frontier_size

        # Update max frontier size during each depth iteration
        max_frontier_size = max(max_frontier_size, depth)  # Update based on depth iteration


    return None, max_frontier_size

In [237]:
# Running IDS with verbose output
actions, cost = iterative_deepening_search(problem, verbose=True)
print(f"Solution(actions, step_cost): {actions}, \nMax Frontier Size: {cost}")

Frontier at step 1

 1 2 3 
 4 5   
 7 8 6 

 1 2 3 
 4 5 6 
 7   8 
----------------------------------------------------------------------------------------------------
Frontier at step 2

 1 2 3 
 4 5   
 7 8 6 

 1 2 3 
 4 5 6 
 7   8 
----------------------------------------------------------------------------------------------------
Frontier at step 3

 1 2   
 4 5 3 
 7 8 6 

 1 2 3 
 4 5 6 
 7 8   

 1 2 3 
 4   5 
 7 8 6 
----------------------------------------------------------------------------------------------------
Frontier at step 4

 1 2 3 
 4   6 
 7 5 8 

 1 2 3 
 4 5 6 
   7 8 

 1 2 3 
 4 5 6 
 7 8   
----------------------------------------------------------------------------------------------------
Frontier at step 5

 1 2 3 
 4 5   
 7 8 6 

 1 2 3 
 4 5 6 
 7   8 
----------------------------------------------------------------------------------------------------
Frontier at step 6

 1 2   
 4 5 3 
 7 8 6 

 1 2 3 
 4 5 6 
 7 8   

 1 2 3 
 4   5 
 7 8 6 
------

In [238]:
def uniform_cost_search(problem, verbose=False):
    """
    Implements Uniform Cost Search (UCS) for the 8-puzzle problem.
    """
    node = Node.root(problem.init_state)  # Create the root node from the initial state
    if problem.goal_test(node.state):  # Check if the initial state is the goal
        return solution(node), 1  # Return immediately if start is the goal

    frontier = deque([node])  # Initialize the frontier with the root node
    explored = set()  # Set to keep track of explored nodes
    max_frontier_size = 1  # Initialize max frontier size

    if verbose:
        visualizer = Visualizer(problem)  # Optional visualizer for debugging
        visualizer.visualize(frontier)

    while frontier:  # While there are nodes to explore
        node = frontier.popleft()  # Dequeue the first node in the frontier
        explored.add(node.state)  # Mark the node as explored

        # Expand the frontier with the children
        for action in problem.actions(node.state):  # Get possible actions from the current state
            child = Node.child(problem, node, action)  # Generate child node
            if child.state not in explored and child not in frontier:  # Check if child is unexplored
                if problem.goal_test(child.state):  # Check if the child is the goal
                    return solution(child), max_frontier_size  # Return the solution and max frontier size
                frontier.append(child)  # Add child to the frontier

        # Track the maximum size of the frontier
        max_frontier_size = max(max_frontier_size, len(frontier))

        if verbose:
            visualizer.visualize(frontier)  # Visualize the current state of the frontier

    return None, max_frontier_size  # Return None if no solution is found


In [239]:
# Running UCS with verbose output
actions, cost = uniform_cost_search(problem, verbose=True)
print(f"Solution(actions, step_cost): {actions}, \nMax Frontier Size: {cost}")

Frontier at step 1

 1 2 3 
 4 5 6 
 7 8   
----------------------------------------------------------------------------------------------------
Frontier at step 2

 1 2 3 
 4 5   
 7 8 6 

 1 2 3 
 4 5 6 
 7   8 
----------------------------------------------------------------------------------------------------
Frontier at step 3

 1 2 3 
 4 5 6 
 7   8 

 1 2   
 4 5 3 
 7 8 6 

 1 2 3 
 4   5 
 7 8 6 
----------------------------------------------------------------------------------------------------
Frontier at step 4

 1 2   
 4 5 3 
 7 8 6 

 1 2 3 
 4   5 
 7 8 6 

 1 2 3 
 4   6 
 7 5 8 

 1 2 3 
 4 5 6 
   7 8 
----------------------------------------------------------------------------------------------------
Frontier at step 5

 1 2 3 
 4   5 
 7 8 6 

 1 2 3 
 4   6 
 7 5 8 

 1 2 3 
 4 5 6 
   7 8 

 1   2 
 4 5 3 
 7 8 6 
----------------------------------------------------------------------------------------------------
Frontier at step 6

 1 2 3 
 4   6 
 7 5 8 

 1 2 

In [240]:
# Heuristic function
def heuristic(state, goal_state):
    '''Heuristic function: counts the number of misplaced tiles.'''
    return sum(1 if state[i] != goal_state[i] else 0 for i in range(len(state)))

# Stochastic Hill Climbing Algorithm
def stochastic_hill_climbing(problem, verbose=False):
    '''Function to perform Stochastic Hill Climbing on the sliding puzzle.'''
    current = Node.root(problem.init_state)

    print("Initial state:")
    sliding_puzzle_visualizer(problem, current.state)

    while not problem.goal_test(current.state):
        # Get all neighbors
        neighbors = [Node.child(problem, current, action) for action in problem.actions(current.state)]

        # Filter for neighbors with a lower heuristic than the current state
        next_node = min(neighbors, key=lambda node: heuristic(node.state, problem._goal_state))

        if heuristic(next_node.state, problem._goal_state) >= heuristic(current.state, problem._goal_state):
            break  # No better neighbors, exit

        current = next_node
        sliding_puzzle_visualizer(problem, current.state)

    return current

# Running stochastic hill climbing
goal_node = stochastic_hill_climbing(problem)
print("Final state reached:")
sliding_puzzle_visualizer(problem, goal_node.state)
actions, cost = solution(goal_node)
print("Solution actions:", actions)
print("Cost of the solution:", cost)

Initial state:
 1 2 3 
 4 5 6 
 7 8   
 1 2 3 
 4 5 6 
 7   8 
 1 2 3 
 4 5 6 
   7 8 
 1 2 3 
   5 6 
 4 7 8 
Final state reached:
 1 2 3 
   5 6 
 4 7 8 
Solution actions: ['left', 'left', 'up']
Cost of the solution: 3


In [241]:
import math
import random

# Simulated annealing function
def simulated_annealing(problem, schedule, verbose=False, max_steps=12):
    """Simulated annealing search implementation."""
    current_node = Node.root(problem.init_state)  # Initialize with the root node
    current_value = problem.heuristic(current_node.state)  # Evaluate the heuristic

    if verbose:
        visualizer = Visualizer(problem)  # Initialize visualizer if verbose is enabled

    for t in range(max_steps):  # Run for a fixed number of steps
        if verbose:
            visualizer.visualize([current_node])  # Visualize current state

        T = schedule(t)  # Get temperature based on step count
        if current_value == 0:  # Goal check (if heuristic is 0, goal is reached)
            return current_node  # Return the node instead of just the state
        if T == 0:  # Stop if temperature reaches 0 (shouldn't happen too early)
            break

        # Generate the next state by choosing a random valid action
        next_actions = problem.actions(current_node.state)
        action = random.choice(next_actions)  # Randomly pick an action
        next_node = Node.child(problem, current_node, action)  # Generate child node
        next_value = problem.heuristic(next_node.state)  # Calculate heuristic for the new state
        delta = current_value - next_value  # Difference in heuristic values

        # If the next state is better, or accept with some probability depending on temperature
        if delta > 0 or random.random() < math.exp(delta / T):
            current_node, current_value = next_node, next_value  # Update to the new state

    return current_node  # Return the final node after max_steps


# Node class representing nodes in the search tree
class Node:
    def __init__(self, state, parent=None, action=None):
        self.state = state
        self.parent = parent
        self.action = action

    @classmethod
    def root(cls, state):
        return cls(state)

    @classmethod
    def child(cls, problem, parent, action):
        state = problem.result(parent.state, action)
        return cls(state, parent, action)


# SlidingPuzzle problem class
class SlidingPuzzle:
    def __init__(self, init_state, goal_state):
        self.init_state = init_state
        self.goal_state = goal_state

    def actions(self, state):
        """Return a list of valid actions for the given state (e.g., up, down, left, right moves)."""
        empty_idx = state.index(' ')
        possible_actions = []

        # Determine the row and column of the empty tile
        row, col = divmod(empty_idx, 3)

        # Check the possible moves (up, down, left, right) based on the position of the empty tile
        if row > 0:  # Can move up
            possible_actions.append('up')
        if row < 2:  # Can move down
            possible_actions.append('down')
        if col > 0:  # Can move left
            possible_actions.append('left')
        if col < 2:  # Can move right
            possible_actions.append('right')

        return possible_actions

    def result(self, state, action):
        """Return the new state after applying the given action."""
        empty_idx = state.index(' ')
        new_state = state[:]
        row, col = divmod(empty_idx, 3)

        # Determine the new index of the empty space after the action
        if action == 'up':
            swap_idx = empty_idx - 3
        elif action == 'down':
            swap_idx = empty_idx + 3
        elif action == 'left':
            swap_idx = empty_idx - 1
        elif action == 'right':
            swap_idx = empty_idx + 1

        # Swap the empty tile with the adjacent tile
        new_state[empty_idx], new_state[swap_idx] = new_state[swap_idx], new_state[empty_idx]

        return new_state

    def heuristic(self, state):
        """Simple heuristic: number of tiles out of place compared to the goal state."""
        return sum(1 for i in range(len(state)) if state[i] != self.goal_state[i] and state[i] != ' ')


# Visualizer class for optional visualization
class Visualizer:
    def __init__(self, problem):
        self.problem = problem

    def visualize(self, nodes):
        """Visualize the state or progression of the algorithm."""
        for node in nodes:
            sliding_puzzle_visualizer(self.problem, node.state)


# Function to extract the solution path from the final node
def solution(node):
    actions = []
    cost = 0
    while node.parent is not None:
        actions.append(node.action)
        node = node.parent
        cost += 1
    actions.reverse()  # Reverse the actions to get them from start to goal
    return actions, cost


# Helper function to visualize the final state
def sliding_puzzle_visualizer(problem, state):
    """Simple visualization of the sliding puzzle state."""
    for i in range(0, 9, 3):
        print(state[i:i+3])  # Print the 3x3 puzzle grid


# Schedule function (linear cooling)
def schedule(t):
    return max(1e-4, 1 - 0.01 * t)  # Temperature decreases as t increases


# Initial and goal state for the sliding puzzle
init_state = ['1', '2', '3', '4', '5', '6', '7', '8', ' ']  # Initial state
goal_state = ['1', '2', '3', ' ', '5', '6', '4', '7', '8']  # Goal state

# Create the sliding puzzle problem instance
puzzle = SlidingPuzzle(init_state, goal_state)

# Run simulated annealing for the sliding puzzle
goal_node = simulated_annealing(puzzle, schedule, verbose=True, max_steps=1000)

# Visualize the final state and output
print("\nFinal State:")
sliding_puzzle_visualizer(puzzle, goal_node.state)

# Get the solution actions and cost
actions, cost = solution(goal_node)
print("Solution actions:", actions)
print("Cost of the solution:", cost)


['1', '2', '3']
['4', '5', '6']
['7', '8', ' ']
['1', '2', '3']
['4', '5', '6']
['7', ' ', '8']
['1', '2', '3']
['4', '5', '6']
['7', ' ', '8']
['1', '2', '3']
['4', '5', '6']
['7', '8', ' ']
['1', '2', '3']
['4', '5', '6']
['7', '8', ' ']
['1', '2', '3']
['4', '5', '6']
['7', '8', ' ']
['1', '2', '3']
['4', '5', '6']
['7', '8', ' ']
['1', '2', '3']
['4', '5', ' ']
['7', '8', '6']
['1', '2', '3']
['4', '5', ' ']
['7', '8', '6']
['1', '2', '3']
['4', ' ', '5']
['7', '8', '6']
['1', '2', '3']
['4', '8', '5']
['7', ' ', '6']
['1', '2', '3']
['4', '8', '5']
[' ', '7', '6']
['1', '2', '3']
['4', '8', '5']
[' ', '7', '6']
['1', '2', '3']
[' ', '8', '5']
['4', '7', '6']
['1', '2', '3']
['8', ' ', '5']
['4', '7', '6']
['1', ' ', '3']
['8', '2', '5']
['4', '7', '6']
['1', ' ', '3']
['8', '2', '5']
['4', '7', '6']
['1', ' ', '3']
['8', '2', '5']
['4', '7', '6']
['1', ' ', '3']
['8', '2', '5']
['4', '7', '6']
['1', ' ', '3']
['8', '2', '5']
['4', '7', '6']
['1', ' ', '3']
['8', '2', '5']
['4', '7

In [242]:
import random

# Define the SlidingPuzzle class to encapsulate the puzzle logic
class SlidingPuzzle:
    def __init__(self, init_state, goal_state):
        self.init_state = init_state
        self.goal_state = goal_state

    # Define the available actions based on the current state of the puzzle
    def actions(self, state):
        empty_index = state.index(' ')
        possible_actions = []
        if empty_index not in [0, 1, 2]:  # Can move 'up'
            possible_actions.append('up')
        if empty_index not in [6, 7, 8]:  # Can move 'down'
            possible_actions.append('down')
        if empty_index not in [0, 3, 6]:  # Can move 'left'
            possible_actions.append('left')
        if empty_index not in [2, 5, 8]:  # Can move 'right'
            possible_actions.append('right')
        return possible_actions

    # Resulting state after applying an action
    def result(self, state, action):
        state = list(state)  # Convert string to list for mutability
        empty_index = state.index(' ')
        if action == 'up':
            new_index = empty_index - 3
        elif action == 'down':
            new_index = empty_index + 3
        elif action == 'left':
            new_index = empty_index - 1
        elif action == 'right':
            new_index = empty_index + 1
        # Swap the empty tile with the tile in the new position
        state[empty_index], state[new_index] = state[new_index], state[empty_index]
        return ''.join(state)

    # Heuristic: Number of misplaced tiles
    def heuristic(self, state):
        return sum(1 for i, tile in enumerate(state) if tile != ' ' and tile != self.goal_state[i])

# Recombination of two parents
def recombine(x, y):
    n = len(x)
    c = random.randrange(0, n)
    return x[:c] + y[c:]

# Mutate an individual by changing a random gene
def mutate(x, gene_pool, pmut):
    if random.uniform(0, 1) >= pmut:
        return x
    n = len(x)
    c = random.randrange(0, n)  # choose position to mutate
    r = random.randrange(0, len(gene_pool))  # choose new random gene from pool
    return x[:c] + [gene_pool[r]] + x[c+1:]

# Selection function to pick parents based on fitness
def select(k, population, fitness_fn):
    fitness_values = [max(fitness_fn(ind), 1) for ind in population]  # Ensures non-zero weights
    return random.choices(
        population=population,
        weights=fitness_values,
        k=k
    )

# Fitness function for the sliding puzzle
def fitness_fn(individual, problem):
    # Start at the initial state
    node = Node.root(problem.init_state)

    # Simulate the moves (actions) in the individual
    for action in individual:
        if action in problem.actions(node.state):
            node = Node.child(problem, node, action)
        else:
            break  # If an invalid action is selected, stop

    # If the state matches the goal, return a large fitness value
    if node.state == problem.goal_state:
        return 1000  # Large fitness for reaching the goal state

    # Otherwise, return the negative heuristic value (number of misplaced tiles)
    return -problem.heuristic(node.state)

# Function to visualize the sliding puzzle state
def visualize_puzzle(state):
    """Function to display the current puzzle state in a readable format."""
    for i in range(0, 9, 3):
        print(' ' + ' '.join(state[i:i + 3]) + ' ')
    print()  # New line for better readability

# Genetic algorithm
def genetic_algorithm(problem, gene_pool, ngen=1000, pmut=0.1, population_size=100, action_sequence_length=50):
    # Generate an initial population with random action sequences
    population = [[random.choice(gene_pool) for _ in range(action_sequence_length)] for _ in range(population_size)]

    for generation in range(ngen):
        # Create the next generation through recombination and mutation
        population = [mutate(recombine(*select(2, population, lambda ind: fitness_fn(ind, problem))), gene_pool, pmut)
                      for _ in range(len(population))]

        # Find the fittest individual in the current generation
        fittest_individual = max(population, key=lambda ind: fitness_fn(ind, problem))

        # Check if a solution is found
        if fitness_fn(fittest_individual, problem) > 999:  # Solution fitness = 1000
            print(f"Solution found at generation {generation + 1}")
            # Visualize the steps to the solution
            simulate_solution(problem, fittest_individual)  # Visualize the steps
            return fittest_individual

        # Print the status every 100 generations
        if generation % 100 == 0:
            print(f"Generation {generation}, best fitness: {fitness_fn(fittest_individual, problem)}")

    # Return the best individual if no exact solution is found
    return max(population, key=lambda ind: fitness_fn(ind, problem))

# Function to simulate the steps of the solution
def simulate_solution(problem, action_sequence):
    """Simulate the moves in the action sequence and visualize the puzzle at each step."""
    node = Node.root(problem.init_state)
    print("Initial state:")
    visualize_puzzle(node.state)

    for action in action_sequence:
        if action in problem.actions(node.state):
            node = Node.child(problem, node, action)
            print(f"After action '{action}':")
            visualize_puzzle(node.state)

    # Final state
    print("Goal state reached:")
    visualize_puzzle(node.state)

Start_time = time.time()

# Define the gene pool (possible actions)
gene_pool = ['up', 'down', 'left', 'right']

# Define the problem instance
problem = SlidingPuzzle('12345678 ', '123 56478')

# Run the genetic algorithm
solution = genetic_algorithm(problem, gene_pool)

# Display the final solution (action sequence) and its cost
actions, cost = solution, len(solution)

Elapsed_Time = time.time() - Start_time

Genetic_actions, Genetic_cost = actions, cost

print(f"Solution actions: {actions}")
print(f"Solution cost: {cost}")

Solution found at generation 1
Initial state:
 1 2 3 
 4 5 6 
 7 8   

After action 'left':
 1 2 3 
 4 5 6 
 7   8 

After action 'left':
 1 2 3 
 4 5 6 
   7 8 

After action 'up':
 1 2 3 
   5 6 
 4 7 8 

After action 'right':
 1 2 3 
 5   6 
 4 7 8 

After action 'down':
 1 2 3 
 5 7 6 
 4   8 

After action 'up':
 1 2 3 
 5   6 
 4 7 8 

After action 'up':
 1   3 
 5 2 6 
 4 7 8 

After action 'down':
 1 2 3 
 5   6 
 4 7 8 

After action 'up':
 1   3 
 5 2 6 
 4 7 8 

After action 'down':
 1 2 3 
 5   6 
 4 7 8 

After action 'down':
 1 2 3 
 5 7 6 
 4   8 

After action 'left':
 1 2 3 
 5 7 6 
   4 8 

After action 'right':
 1 2 3 
 5 7 6 
 4   8 

After action 'right':
 1 2 3 
 5 7 6 
 4 8   

After action 'left':
 1 2 3 
 5 7 6 
 4   8 

After action 'left':
 1 2 3 
 5 7 6 
   4 8 

After action 'right':
 1 2 3 
 5 7 6 
 4   8 

After action 'left':
 1 2 3 
 5 7 6 
   4 8 

After action 'up':
 1 2 3 
   7 6 
 5 4 8 

After action 'right':
 1 2 3 
 7   6 
 5 4 8 

After action '

In [243]:
from heapq import heappop, heappush

class Problem:
    def __init__(self):
        self.init_state = None

    def actions(self, state):
        raise NotImplementedError

    def result(self, state, action):
        raise NotImplementedError

    def goal_test(self, state):
        raise NotImplementedError

    def step_cost(self, state, action):
        raise NotImplementedError


class Node:
    def __init__(self, state, parent, action, path_cost):
        self.state = state
        self.parent = parent
        self.action = action
        self.path_cost = path_cost

    @classmethod
    def root(cls, init_state):
        return cls(init_state, None, None, 0)

    @classmethod
    def child(cls, problem, parent, action):
        return cls(
            problem.result(parent.state, action),
            parent,
            action,
            parent.path_cost + problem.step_cost(parent.state, action))


def solution(node):
    actions = []
    cost = node.path_cost
    while node.parent is not None:
        actions.append(node.action)
        node = node.parent
    actions.reverse()
    return actions, cost


class SlidingPuzzle(Problem):
    def __init__(self, init_state, goal_state):
        assert init_state.count(' ') == 1
        assert goal_state.count(' ') == 1
        self.init_state = tuple(init_state)
        self._goal_state = tuple(goal_state)
        self._action_values = {'up': -3, 'down': +3, 'left': -1, 'right': +1}

    def actions(self, state):
        index = state.index(' ')
        possible_moves = []
        if index // 3 > 0:
            possible_moves.append('up')
        if index // 3 < 2:
            possible_moves.append('down')
        if index % 3 > 0:
            possible_moves.append('left')
        if index % 3 < 2:
            possible_moves.append('right')
        return possible_moves

    def result(self, state, action):
        def swap(t, i, j):
            l = list(t)
            l[i], l[j] = l[j], l[i]
            return tuple(l)
        index = state.index(' ')
        return swap(state, index, index + self._action_values[action])

    def goal_test(self, state):
        return state == self._goal_state

    def step_cost(self, state, action):
        return 1

    def heuristic(self, state):
        """Simple heuristic: number of tiles out of place compared to the goal state."""
        return sum(1 for i in range(len(state)) if state[i] != self._goal_state[i])


def print_puzzle_state(state):
    """Prints the puzzle state in a 3x3 grid."""
    for i in range(0, 9, 3):
        print(' ' + ' '.join(state[i:i + 3]) + ' ')
    print()


def greedy_best_first_graph_search(problem, h=None, verbose=False):
    """Greedy best-first search with optional verbose output."""
    h = h or problem.heuristic  # Use the provided heuristic function, or the problem's default one.
    node = Node.root(problem.init_state)
    frontier = [(h(node.state), node)]  # Priority queue ordered by h(n)
    explored = set()
    max_frontier_size = 1  # Track the maximum size of the frontier

    if verbose:
        print(f"Initial state:")
        print_puzzle_state(node.state)

    while frontier:
        _, node = heappop(frontier)

        # Show action and cost of the current node
        if node.parent is not None and verbose:
            print(f"Action: {node.action}, Path cost: {node.path_cost}")
            print_puzzle_state(node.state)

        if problem.goal_test(node.state):
            return solution(node), max_frontier_size

        explored.add(node.state)
        for action in problem.actions(node.state):
            child = Node.child(problem, node, action)
            if child.state not in explored and all(c.state != child.state for _, c in frontier):
                heappush(frontier, (h(child.state), child))
                # Show action and cost when adding the node to the frontier
                if verbose:
                    print(f"Adding to frontier: Action: {action}, Path cost: {child.path_cost}")
                    print_puzzle_state(child.state)

                if problem.goal_test(child.state):
                    return solution(child), max_frontier_size

        max_frontier_size = max(max_frontier_size, len(frontier))

    return None, max_frontier_size


# Example to solve the sliding puzzle
problem = SlidingPuzzle('12345678 ', '123 56478')

# Run the greedy best-first search with verbose output
solution_steps, max_frontier_size = greedy_best_first_graph_search(problem, verbose=True)

if solution_steps:
    print("\nSolution found!")
    print(f"Steps to solution:")
    for action in solution_steps[0]:
        print(action)
    print(f"Total cost: {solution_steps[1]}")
else:
    print("No solution found.")


Initial state:
 1 2 3 
 4 5 6 
 7 8   

Adding to frontier: Action: up, Path cost: 1
 1 2 3 
 4 5   
 7 8 6 

Adding to frontier: Action: left, Path cost: 1
 1 2 3 
 4 5 6 
 7   8 

Action: left, Path cost: 1
 1 2 3 
 4 5 6 
 7   8 

Adding to frontier: Action: up, Path cost: 2
 1 2 3 
 4   6 
 7 5 8 

Adding to frontier: Action: left, Path cost: 2
 1 2 3 
 4 5 6 
   7 8 

Action: left, Path cost: 2
 1 2 3 
 4 5 6 
   7 8 

Adding to frontier: Action: up, Path cost: 3
 1 2 3 
   5 6 
 4 7 8 


Solution found!
Steps to solution:
left
left
up
Total cost: 3


In [244]:
from heapq import heappop, heappush

def memoize(fn, slot=None):
    """Memoize function `fn`: store results in `slot` if provided, or in `cache`."""
    if slot:
        def memoized_fn(obj, *args):
            if not hasattr(obj, slot):
                setattr(obj, slot, {})
            cache = getattr(obj, slot)
            if args not in cache:
                cache[args] = fn(obj, *args)
            return cache[args]
    else:
        def memoized_fn(*args):
            if not hasattr(memoized_fn, 'cache'):
                memoized_fn.cache = {}
            if args not in memoized_fn.cache:
                memoized_fn.cache[args] = fn(*args)
            return memoized_fn.cache[args]
    return memoized_fn

def astar_search(problem, h=None, verbose=False):
    """A* search with optional verbose output."""
    h = memoize(h or problem.heuristic)  # Memoize the heuristic function for efficiency.
    node = Node.root(problem.init_state)
    frontier = [(h(node.state) + node.path_cost, 0, node)]  # Priority queue ordered by f(n) = g(n) + h(n)
    explored = set()
    max_frontier_size = 1  # Track the maximum size of the frontier

    visualizer = Visualizer(problem) if verbose else None

    if verbose:
        print("Initial state:")
        print_puzzle_state(node.state)

    while frontier:
        _, _, node = heappop(frontier)

        # If goal is reached, return the solution
        if problem.goal_test(node.state):
            return solution(node), max_frontier_size

        explored.add(node.state)
        for action in problem.actions(node.state):
            child = Node.child(problem, node, action)
            f = child.path_cost + h(child.state)  # f(n) = g(n) + h(n)
            if child.state not in explored and all(c.state != child.state for _, _, c in frontier):
                heappush(frontier, (f, len(explored), child))

                if verbose:
                    print(f"Action: {action}, Path cost: {child.path_cost}, f(n): {f}")
                    print_puzzle_state(child.state)

                # If goal is reached in this child state, return the solution
                if problem.goal_test(child.state):
                    return solution(child), max_frontier_size

        max_frontier_size = max(max_frontier_size, len(frontier))

        if verbose:
            visualizer.visualize([n for _, _, n in frontier])

    return None, max_frontier_size

# Assuming SlidingPuzzle and other classes (Node, Visualizer, etc.) are already defined
# Let's define a helper function to print puzzle states.

def print_puzzle_state(state):
    """Prints the puzzle state in a 3x3 grid."""
    for i in range(0, 9, 3):
        print(' ' + ' '.join(state[i:i + 3]) + ' ')
    print()

# Example to solve the sliding puzzle using A* search
problem = SlidingPuzzle('12345678 ', '123 56478')

# Run the A* search with verbose output to see steps
solution_steps, max_frontier_size = astar_search(problem, verbose=True)

if solution_steps:
    print("\nSolution found!")
    print(f"Steps to solution:")
    for action in solution_steps[0]:
        print(action)
    print(f"Total cost: {solution_steps[1]}")
else:
    print("No solution found.")


Initial state:
 1 2 3 
 4 5 6 
 7 8   

Action: up, Path cost: 1, f(n): 6
 1 2 3 
 4 5   
 7 8 6 

Action: left, Path cost: 1, f(n): 4
 1 2 3 
 4 5 6 
 7   8 

('1', '2', '3')
('4', '5', '6')
('7', ' ', '8')
('1', '2', '3')
('4', '5', ' ')
('7', '8', '6')
Action: up, Path cost: 2, f(n): 6
 1 2 3 
 4   6 
 7 5 8 

Action: left, Path cost: 2, f(n): 4
 1 2 3 
 4 5 6 
   7 8 

('1', '2', '3')
('4', '5', '6')
(' ', '7', '8')
('1', '2', '3')
('4', ' ', '6')
('7', '5', '8')
('1', '2', '3')
('4', '5', ' ')
('7', '8', '6')
Action: up, Path cost: 3, f(n): 3
 1 2 3 
   5 6 
 4 7 8 


Solution found!
Steps to solution:
left
left
up
Total cost: 3


In [245]:
import time

def measure_performance(search_algorithm, problem, verbose=False):
    """
    Wrapper function to measure the time and space performance of the search algorithm.
    """
    # Start measuring time
    start_time = time.time()

    # Run the search algorithm, capturing the solution and max frontier size
    solution_result, max_frontier_size = search_algorithm(problem, verbose=verbose)

    # Measure elapsed time
    elapsed_time = time.time() - start_time

    return {
        'solution': solution_result[0] if solution_result else None,
        'cost': solution_result[1] if solution_result else 0,  # Ensure the cost is captured
        'elapsed_time': elapsed_time,
        'max_frontier_size': max_frontier_size  # Return the max number of nodes in the frontier
    }


In [246]:
def stochastic_performance():
    """
    Wrapper function to measure the time and space performance of the search algorithm.
    """
    # Start measuring time
    start_time = time.time()

    # Run the search algorithm, capturing the solution and max frontier size
    goal_node = stochastic_hill_climbing(problem)
    actions, cost = solution(goal_node)

    # Measure elapsed time
    elapsed_time = time.time() - start_time

    return {
        'solution': actions,
        'cost': cost,  # Ensure the cost is captured
        'elapsed_time': elapsed_time,
        'max_frontier_size': 'None'  # Return the max number of nodes in the frontier
    }

In [247]:
def simulated_annealing_performance():
    """
    Wrapper function to measure the time and space performance of the search algorithm.
    """
    # Start measuring time
    start_time = time.time()

    # Initial and goal state for the sliding puzzle
    init_state = ['1', '2', '3', '4', '5', '6', '7', '8', ' ']  # Initial state
    goal_state = ['1', '2', '3', ' ', '5', '6', '4', '7', '8']  # Goal state

    # Create the sliding puzzle problem instance
    puzzle = SlidingPuzzle(init_state, goal_state)

    # Run simulated annealing for the sliding puzzle
    goal_node = simulated_annealing(puzzle, schedule, verbose=False, max_steps=1000)

    actions, cost = solution(goal_node)

    # Measure elapsed time
    elapsed_time = time.time() - start_time

    return {
        'solution': actions,
        'cost': cost,  # Ensure the cost is captured
        'elapsed_time': elapsed_time,
        'max_frontier_size': 'None'  # Return the max number of nodes in the frontier
    }

In [248]:
# Ensure that the solution is always a list, even if it's a single action
def format_solution(performance):
    solution = performance['solution']
    if isinstance(solution, str):  # If the solution is a string (e.g., 'left'), convert it to a list
        return [solution]
    return solution

# Measure performance for all search algorithms, including BFS, DFS, IDS, UCS, A*, and Greedy
bfs_graph_performance = measure_performance(breadth_first_graph_search, problem, verbose=False)
dfs_graph_performance = measure_performance(depth_first_graph_search, problem, verbose=False)
iterative_deepening_performance = measure_performance(iterative_deepening_search, problem, verbose=False)
uniform_cost_performance = measure_performance(uniform_cost_search, problem, verbose=False)
a_star_performance = measure_performance(astar_search, problem, verbose=False)
greedy_performance = measure_performance(greedy_best_first_graph_search, problem, verbose=False)
stochastic_hill_climbing_performance = stochastic_performance()
Simulated_annealing_performance = simulated_annealing_performance()

# Format the solutions correctly for the table (using the function defined above)
bfs_graph_performance['solution'] = format_solution(bfs_graph_performance)
dfs_graph_performance['solution'] = format_solution(dfs_graph_performance)
iterative_deepening_performance['solution'] = format_solution(iterative_deepening_performance)
uniform_cost_performance['solution'] = format_solution(uniform_cost_performance)
a_star_performance['solution'] = format_solution(a_star_performance)
greedy_performance['solution'] = format_solution(greedy_performance)


# Prepare the data for the extended table
headers = ["Algorithm", "Solution", "Cost", "Time (seconds)", "Max Frontier Size (nodes)"]
table_data = [
    ["BFS Graph", bfs_graph_performance['solution'], bfs_graph_performance['cost'],
     f"{bfs_graph_performance['elapsed_time']:.4f}", bfs_graph_performance['max_frontier_size']],

    ["DFS Graph", dfs_graph_performance['solution'], dfs_graph_performance['cost'],
     f"{dfs_graph_performance['elapsed_time']:.5f}", dfs_graph_performance['max_frontier_size']],

    ["IDS", iterative_deepening_performance['solution'], iterative_deepening_performance['cost'],
     f"{iterative_deepening_performance['elapsed_time']:.4f}", iterative_deepening_performance['max_frontier_size']],

    ["UCS", uniform_cost_performance['solution'], uniform_cost_performance['cost'],
     f"{uniform_cost_performance['elapsed_time']:.4f}", uniform_cost_performance['max_frontier_size']],

    ["A* Search", a_star_performance['solution'], a_star_performance['cost'],
     f"{a_star_performance['elapsed_time']:.4f}", a_star_performance['max_frontier_size']],

    ["Greedy Search", greedy_performance['solution'], greedy_performance['cost'],
     f"{greedy_performance['elapsed_time']:.4f}", greedy_performance['max_frontier_size']],

    ["stochastic_hill_climbing", stochastic_hill_climbing_performance['solution'], stochastic_hill_climbing_performance['cost'],
     f"{stochastic_hill_climbing_performance['elapsed_time']:.4f}", stochastic_hill_climbing_performance['max_frontier_size']],

    ["Simulated_annealing", Simulated_annealing_performance['solution'], Simulated_annealing_performance['cost'],
     f"{Simulated_annealing_performance['elapsed_time']:.4f}", Simulated_annealing_performance['max_frontier_size']],

    ["Genetic performance", Genetic_actions, Genetic_cost,
     f"{Elapsed_Time:.4f}", 'None']
]

# Print the table
from tabulate import tabulate
print(tabulate(table_data, headers=headers, tablefmt='grid'))

Initial state:
('1', '2', '3')
('4', '5', '6')
('7', '8', ' ')
('1', '2', '3')
('4', '5', '6')
('7', ' ', '8')
('1', '2', '3')
('4', '5', '6')
(' ', '7', '8')
('1', '2', '3')
(' ', '5', '6')
('4', '7', '8')
+--------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------+------------------+-----------------------------+
| Algorithm                | Solution                                                                                                                                                                                                                                                                                   